<a href="https://colab.research.google.com/github/denizozturk21/1300ABTesting/blob/master/DeepLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize, sent_tokenize
import string
from nltk.corpus import stopwords
import tensorflow as tf
import numpy as np
from tensorflow.keras import Model


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
def preprocess(filepath):# load data

  print(filepath)

  tokenized_final = []
  file = open(filepath, 'rt')
  text = file.read()
  file.close()

  # remove punctuation from each word but periods
  text = text.translate(str.maketrans('', '', '!"#$%&()*+,-/:;<=>?@[\]^_`{|}~'))


  sentences = sent_tokenize(text)

  tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(sentences, target_vocab_size=2**13)

  #remove newlines and check for sentence length
  for x in range(len(sentences)):
    sentence = sentences[x]
    sentence.replace('\n', ' ')
    if len(sentence.split()) < 8:
      sentences[x] = "?"
  
  print(sentences[:100])
  #tokenize
  tokenized_sents = [word_tokenize(i) for i in sentences]
  flattened_tokeninized_final = [i for j in tokenized_sents for i in j]

  #set all to lower case
  tokens = [w.lower() for w in flattened_tokeninized_final]
  print(tokens[:100])
  
  # remove remaining tokens that are not alphabetic
  words = [word for word in tokens if word.isdigit() == False]

  # print(words[:100])

  # filter out stop words if needed
  # stop_words = set(stopwords.words('english'))
  # words = [w for w in words if not w in stop_words]
  print(words[:100])
  return words, tokenizer

In [ ]:
# quran_filepath = "/content/English-Maulana-Mohammad-Ali-103.txt"
# quran = preprocess(quran_filepath)
import tensorflow_datasets as tfds

bible_filepath = "/content/bible.txt"
bible, tokenizer = preprocess(bible_filepath)


# print(bible[:100])
print(len(bible)) #787849 #820257 with periods



/content/bible.txt
['11 In the beginning God created the heaven and the earth.', '12 And the earth was without form and void and darkness was upon\nthe face of the deep.', 'And the Spirit of God moved upon the face of the\nwaters.', '13 And God said Let there be light and there was light.', '14 And God saw the light that it was good and God divided the light\nfrom the darkness.', '15 And God called the light Day and the darkness he called Night.', 'And the evening and the morning were the first day.', '16 And God said Let there be a firmament in the midst of the waters\nand let it divide the waters from the waters.', '17 And God made the firmament and divided the waters which were\nunder the firmament from the waters which were above the firmament\nand it was so.', '?', 'And the evening and the\nmorning were the second day.', '19 And God said Let the waters under the heaven be gathered together\nunto one place and let the dry land appear and it was so.', '110 And God called the dry lan

In [ ]:
print(len(set(bible))) # 12901

12901


In [ ]:
#print(len(sentences))

I wrote a function to get a dictionary of unqiue terms+key values as well as turning everything into 1-hot vectors. Might be kinda slow, since it runs for loops.

In [ ]:
# :param words: a list of words (strings)
# :return: a dictionary containing all unique words in the string,
# with a corresponding key value, and a 2-D tensor where each row
# is a 1-hot vector.
def dict_and_one_hot(words):
	# create a set from the words, to eliminate duplicates
	wordsList = set(words)

	# initialize our dictionary as well as our key value placeholder
	dict = {}
	i = 0

	# loop through all words in our set, and add them to our dict
	# and set its value to the key value, then increment our key
	# value placeholder by 1.
	for word in wordsList:
		dict[word] = i
		i += 1

	# initialize a list to hold our indices as well as how many
	# different words there are.
	indices = []
	depth = len(dict)

	# loop through all words, and store their key value as a list.
	for word in words:
		indices.append(dict[word])

	# return the dict, as well as applying tf.one_hot to our list
	# of indices.
	return dict, indices #tf.one_hot(indices, depth)

In [ ]:
# function demonstration
import tensorflow as tf
import keras
# d, w = dict_and_one_hot(quran)
d2, w2 = dict_and_one_hot(bible)
print("dictionary: ", len(d2))
print("words as 1-hot vectors: ", w2)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Below is an example of the lemmatization and stemming functions from the nltk library. However, we may not want to lemmatize or stem the words, because we may want to preserve part of speech instead of just the essence of the word. On the other hand, keeping part of speech in may be a dead giveaway, lol.

In [ ]:
# we can also try the lemmatization function
#lemmatizer = nltk.stem.WordNetLemmatizer()

#print('dogs lemmatized is: ', lemmatizer.lemmatize('dogs'))

#print()

# but it's not great, since it treats everything as a noun. As you can see, it
# does nothing to the test sentence
#for word in tokenized_test:
#  print(word, " lemmatized is: ", lemmatizer.lemmatize(word))

#print()

# however, if we specify part of speech, it gets better
#print('created lemmatized with PoS is: ', lemmatizer.lemmatize('created', pos='v'))
# but specifying part of speech is a bit much to ask for

In [ ]:
# we can also try the stem function, but it also has its fallbacks
#ps = nltk.stem.PorterStemmer()

#for word in tokenized_test:
#  print(word, " stemmed is: ", ps.stem(word))

#print()

# looks good, but look what happens with this
#print('programmer stemmed is: ', ps.stem('programmer'))

In [ ]:
class Model(tf.keras.Model):
    def __init__(self, vocab_size):
        """
        The Model class predicts the next words in a sequence.

        :param vocab_size: The number of unique words in the data
        """

        super(Model, self).__init__()

        # TODO: initialize vocab_size, embedding_size
        # I filled in all the inital parameters, not much to explain
        self.vocab_size = vocab_size
        self.window_size = 20 # DO NOT CHANGE!
        self.embedding_size = 50
        self.batch_size = 100
        self.hidden_layer_size = 100
        self.rnn_size = 200
        self.learning_rate = 0.01
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=self.learning_rate)

        # TODO: initialize embeddings and forward pass weights (weights, biases)
        # Note: You can now use tf.keras.layers!
        # - use tf.keras.layers.Dense for feed forward layers: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense
        # - and use tf.keras.layers.GRU or tf.keras.layers.LSTM for your RNN
        # Initialize an embedding matrix of size vocab_size x embedding_size
        self.embedding_matrix = tf.Variable(tf.random.truncated_normal([self.vocab_size, self.embedding_size],stddev=0.1))

        # Initialize an LSTM layer
        self.LSTM = tf.keras.layers.LSTM(self.rnn_size, return_sequences=True, return_state=True)

        # Initialize 3 Dense layers, the last one with output dimension of vocab_size
        self.W1 = tf.keras.layers.Dense(self.hidden_layer_size)

        self.W2 = tf.keras.layers.Dense(self.hidden_layer_size)

        self.W3 = tf.keras.layers.Dense(self.vocab_size)


    # made initial_state default to None, b/c that's what it defaults to for LSTM
    def call(self, inputs, initial_state=None):
        """
        - You must use an embedding layer as the first layer of your network (i.e. tf.nn.embedding_lookup)
        - You must use an LSTM or GRU as the next layer.

        :param inputs: word ids of shape (batch_size, window_size)
        :param initial_state: 2-d array of shape (batch_size, rnn_size) as a tensor
        :return: the batch element probabilities as a tensor, a final_state (Note 1: If you use an LSTM, the final_state will be the last two RNN outputs, 
        Note 2: We only need to use the initial state during generation)
        using LSTM and only the probabilites as a tensor and a final_state as a tensor when using GRU 
        """
        
        #TODO: Fill in
        # Lookup the embeddings
        E = tf.nn.embedding_lookup(self.embedding_matrix, inputs)

        # apply the LSTM layer, and collect the logits as well as the state outputs
        l1, final_state1, final_state2 = self.LSTM(E, initial_state=initial_state)

        # apply the 3 dense layers as well as ReLU and Softmax
        l2 = self.W1(l1)
        l2 = tf.nn.relu(l2)

        l3 = self.W2(l2)
        l3 = tf.nn.relu(l3)

        l4 = self.W3(l3)
        l4 = tf.nn.softmax(l4)

        # return the final logits, as well as the final state
        return l4, (final_state1, final_state2)

    def loss(self, probs, labels):
        """
        Calculates average cross entropy sequence to sequence loss of the prediction
        
        NOTE: You have to use np.reduce_mean and not np.reduce_sum when calculating your loss

        :param logits: a matrix of shape (batch_size, window_size, vocab_size) as a tensor
        :param labels: matrix of shape (batch_size, window_size) containing the labels
        :return: the loss of the model as a tensor of size 1
        """

        #TODO: Fill in
        #We recommend using tf.keras.losses.sparse_categorical_crossentropy
        #https://www.tensorflow.org/api_docs/python/tf/keras/losses/sparse_categorical_crossentropy

        # apply the loss function
        loss = tf.keras.losses.sparse_categorical_crossentropy(labels, probs)

        # return the reduced mean of the loss
        return tf.reduce_mean(loss)


# a function that takes two lists as inputs and shuffles them in the same order.
def shuffler(inputs, labels):
	# Get a range of indices, by using tf.range, then shuffle them
	indices = tf.range(start=0, limit=tf.shape(labels)[0], dtype=tf.int32)
	shuffled_indices = tf.random.shuffle(indices)

	# gather the inputs and labels in the order of the shuffled indices
	final_inputs = tf.gather(inputs, shuffled_indices)
	final_labels = tf.gather(labels, shuffled_indices)

	return final_inputs, final_labels


def train(model, train_inputs, train_labels):
    """
    Runs through one epoch - all training examples.

    :param model: the initilized model to use for forward and backward pass
    :param train_inputs: train inputs (all inputs for training) of shape (num_inputs,)
    :param train_labels: train labels (all labels for training) of shape (num_labels,)
    :return: None
    """
    #TODO: Fill in


    # Since we need a bunch of intervals of size window_size, we determine the largest multiple of window_size
    # that is smaller than the size of the training set, and use that as our ending point
    ending = (len(train_labels) // model.window_size) * model.window_size

    # reshape using the ending we found earlier, so now we can reshape into -1 x window_size
    train_inputs = tf.reshape(train_inputs[:ending], [-1, model.window_size])
    train_labels = tf.reshape(train_labels[:ending], [-1, model.window_size])

    # shuffle training inputs and labels.
    inputs, labels = shuffler(train_inputs, train_labels)

    # loop through all elements of the epoch
    i = 0
    while i < (labels.shape[0] - model.batch_size):
        #print("Training Batch: ", i)
        # take out inputs and labels in batches of batches_size
        batch_inputs = inputs[i:i + model.batch_size]
        batch_labels = labels[i:i + model.batch_size]

        # get the tape
        with tf.GradientTape() as tape:
            # call the model on the inputs
            p, state = model.call(batch_inputs)

            # get the loss from the predictions
            losss = model.loss(p, batch_labels)

        # get the gradients and call the optimizer to optimize them
        gradients = tape.gradient(losss, model.trainable_variables)
        op = model.optimizer
        op.apply_gradients(zip(gradients, model.trainable_variables))

        # increment to the next batch
        i += model.batch_size


def test(model, test_inputs, test_labels):
    """
    Runs through one epoch - all testing examples

    :param model: the trained model to use for prediction
    :param test_inputs: train inputs (all inputs for testing) of shape (num_inputs,)
    :param test_labels: train labels (all labels for testing) of shape (num_labels,)
    :returns: perplexity of the test set
    """
    
    #TODO: Fill in
    #NOTE: Ensure a correct perplexity formula (different from raw loss)

    # Since we need a bunch of intervals of size window_size, we determine the largest multiple of window_size
    # that is smaller than the size of the training set, and use that as our ending point
    ending = (len(test_labels) // model.window_size) * model.window_size

    # reshape using the ending we found earlier, so now we can reshape into -1 x window_size
    test_inputs = tf.reshape(test_inputs[:ending], [-1, model.window_size])
    test_labels = tf.reshape(test_labels[:ending], [-1, model.window_size])

    # shuffle testing inputs and labels
    inputs, labels = shuffler(test_inputs, test_labels)

    i = 0
    loss = 0        # we'll need this for perplexity later
    num_batches = 0 # we'll need this for perplexity later
    while i < (labels.shape[0] - model.batch_size):
        #print("Testing Batch: ", i)
        # take out inputs and labels in batches of batches_size
        batch_inputs = inputs[i:i + model.batch_size]
        batch_labels = labels[i:i + model.batch_size]

        p, state = model.call(batch_inputs)

        # get the loss from the predictions
        loss = loss + model.loss(p, batch_labels)

        # increment to the next batch
        i += model.batch_size
        num_batches += 1 # count the number of batches

    # here, we average the average losses to get the total loss for perplexity
    perplexity = loss / num_batches

    # perform the last operation to calculate perplexity (exponential operation) and return the result.
    return np.exp(perplexity)

In [ ]:

model = Model(len(d))

test_index = np.floor(0.9 * len(quran))
test_index = int(test_index)

#print(len(w))

quran_train = tf.cast(w[:test_index], dtype=tf.int32)
quran_test = tf.cast(w[test_index:], dtype=tf.int32)

for _ in range(5):
  train(model, quran_train[:-1], quran_train[1:])

  p = test(model, quran_test[:-1], quran_test[1:])

  print(p)

NameError: ignored

In [ ]:
#generation stuff
words = np.array([[np.random.randint(len(d))]])
for i in range(0,15):
  temp, _ = model.call(words)
  temp = temp[0]
  temp = np.cumsum(temp[-1])
  r = np.random.rand()
  next = np.where(temp > r)
  words = np.array([np.append(words, next[0][0])])

inv_map = {v: k for k, v in d.items()}

for word in words[0]:
  print(inv_map[word])

forbade
war
in
all
things
has
indeed
among
me
say
why
serve
be
blown
if
you


In [ ]:
model2 = Model(len(d2))
for e in range(4):

  test_index = np.floor(0.9 * len(w2))
  test_index = int(test_index)

  #print(len(w2))

  bible_train = tf.cast(w2[:test_index], dtype=tf.int32)
  bible_test = tf.cast(w2[test_index:], dtype=tf.int32)

  train(model2, bible_train[:-1], bible_train[1:])

  p = test(model2, bible_test[:-1], bible_test[1:])

  print(p)

315.21527
270.6853
255.89224
257.45886


In [ ]:
# Generates a sentence from by randomly selecting the next word from the 
# probability distribution described by the logits.
# :param model: a neural net model
# :param seed_word: an array of indices of words
# :param num_words: how many words we want to generate
# :param dictionary: the dictionary containing the vocabulary and its keys and 
# values
# :return: a string representing a sentence.
def generate_rand(model, seed_word, num_words, dictionary):
  # intialize our sentence to be just the words given
  sentence = seed_word
  # loop through the number of words we want.
  for i in range(0, num_words):
    # call the model to get the words
    temp, _ = model.call(sentence)
    # get the last prediction
    temp = temp[0][-1]
    # split the probabilities into bins, choose a random number.
    temp = np.cumsum(temp)
    r = np.random.rand()
    # find the bin the random number falls into, i.e which index in the cumsum
    # is the first to be larger than our random number.
    next = np.where(temp > r)[0][0]
    # append our predicted word index to the sentence.
    sentence = np.array([np.append(sentence, next)])

  # initialize the text we will return as well as create a dictionary that will
  # transform key values into strings.
  text = []
  inv_map = {v: k for k, v in dictionary.items()}

  # loop through all words in the sentence and append the string corresponding
  # to the index to our list of strings
  for word in sentence[0]:
    text.append(inv_map[word])

  # return our string.
  return " ".join(text)

In [ ]:
# Generates a sentence from by randomly selecting the next word from the 
# group of top n choices from the probability distribution described by the
# logits.
# :param model: a neural net model
# :param seed_word: an array of indices of words
# :param num_words: how many words we want to generate
# :param dictionary: the dictionary containing the vocabulary and its keys and 
# values
# :return: a string representing a sentence.
def generate_top(model, seed_word, num_words, dictionary, top):
  # intialize our sentence to be just the words given
  sentence = seed_word
  # loop through the number of words we want.
  top = -1 * top
  for i in range(0, num_words):
    # call the model to get the words
    temp, _ = model.call(sentence)
    # get the last prediction
    temp = temp[0][-1]
    temp = np.array(temp)
    inds = np.argsort(temp)[top:]
    args = temp[inds]
    # split the probabilities into bins, choose a random number.
    args = np.cumsum(args / np.sum(args))
    r = np.random.rand()
    # find the bin the random number falls into, i.e which index in the cumsum
    # is the first to be larger than our random number.
    next = inds[np.where(args > r)[0][0]]
    # append our predicted word index to the sentence.
    sentence = np.array([np.append(sentence, next)])

  # initialize the text we will return as well as create a dictionary that will
  # transform key values into strings.
  text = []
  inv_map = {v: k for k, v in dictionary.items()}

  # loop through all words in the sentence and append the string corresponding
  # to the index to our list of strings
  for word in sentence[0]:
    text.append(inv_map[word])

  # return our string.
  return " ".join(text)

In [ ]:
#generation stuff
words1 = np.array([[np.random.randint(len(d2))]])
print(generate_top(model2, words1, 30, d2, 10))
words = np.array([[np.random.randint(len(d))]])
print(generate_top(model, words, 15, d, 10))

began to reign over him saying the king of israel had brought out the people in their ears . then the son of nethaniah went up into their place and his
arelites confessed blasphemer curdled compound compound composition secondarily sabaoth songs therefore fierce songs storehouse fierce hammothdor


In [ ]:
def text_generator(_model, start, temperature=1.0):
    result = start
    output = [tokenizer.vocab_size] + tokenizer.encode(start)
    output = tf.expand_dims(output, 0)  # Expand by adding batch and time dimensions.
    for i in range(50):
        _model.reset_states()
        prediction = _model(output)
        print(prediction)
        print("hi")
        prediction = tf.squeeze(prediction, 0)
        print("hi")

        prediction = prediction / temperature
        print("hi")
        prediction = tf.random.categorical(prediction, num_samples=1)[-1, 0]
        prediction = tf.squeeze(prediction).numpy()

        if prediction == tokenizer.vocab_size+1 or prediction == 0:
            return result.strip()

        result += tokenizer.decode([prediction])
        prediction = tf.expand_dims([prediction], 0)
        output = tf.concat([output, prediction], axis=-1)

    return result.strip()
print(text_generator(model, 'I '))

(<tf.Tensor: shape=(1, 2, 12901), dtype=float32, numpy=
array([[[7.7452576e-05, 7.7491954e-05, 7.7552948e-05, ...,
         7.7479999e-05, 7.7491699e-05, 7.7506324e-05],
        [7.7500648e-05, 7.7550692e-05, 7.7502336e-05, ...,
         7.7505159e-05, 7.7513549e-05, 7.7504701e-05]]], dtype=float32)>, (<tf.Tensor: shape=(1, 200), dtype=float32, numpy=
array([[ 1.01817474e-02, -6.60065189e-03, -1.54333387e-03,
        -3.66778113e-04, -1.58001762e-02, -4.41129971e-03,
         8.89913063e-04, -2.63793929e-03,  6.08793506e-03,
        -1.99704082e-03,  6.05016714e-03, -2.96840817e-03,
        -6.50398759e-03,  1.62763335e-02,  9.09570698e-03,
        -1.61726214e-02,  3.97202140e-03, -1.72349170e-03,
        -5.29166544e-03,  6.51297392e-03, -1.19396653e-02,
         2.66681723e-02,  1.40450138e-03,  9.87626519e-03,
         4.12335759e-03,  2.50258949e-03, -1.26139317e-02,
         5.48283383e-03, -1.19053209e-02,  3.04993382e-03,
         1.81498360e-02, -5.41173294e-03,  4.45675664e-0

InvalidArgumentError: ignored